# Functions


In [1]:
import dataclasses
from datetime import date


@dataclasses.dataclass
class RehearsalSlot:
    """Represents a single, structured rehearsal event."""
    rehearsal_date: date
    day_of_week: str  # Standardized, lowercase e.g., 'wednesday'
    start_time: int   # Military time, e.g., 1100
    end_time: int     # Military time, e.g., 1200


In [2]:
import dataclasses
from abc import ABC, abstractmethod
from typing import TypeAlias


TimeInterval: TypeAlias = tuple[int, int]

class UnavailabilityConstraint(ABC):
    """Abstract base class for all dancer unavailability constraints."""
    @abstractmethod
    def get_conflicting_intervals(self, slot: RehearsalSlot) -> list[TimeInterval]:
        pass

@dataclasses.dataclass(frozen=True)
class DayOfWeekConstraint(UnavailabilityConstraint):
    """Represents an unavailability for an entire day of the week."""
    day_of_week: str

    def get_conflicting_intervals(self, slot: RehearsalSlot) -> list[TimeInterval]:
        if self.day_of_week == slot.day_of_week:
            return [(slot.start_time, slot.end_time)]
        return []


@dataclasses.dataclass(frozen=True)
class TimeOnDayConstraint(UnavailabilityConstraint):
    """
    Represents an unavailability for a specific time interval on a given day of the week.
    e.g., "Mondays from 900 to 1200"
    """
    day_of_week: str  # e.g., 'monday'
    start_time: int   # Military time, e.g., 900
    end_time: int     # Military time, e.g., 1200

    def get_conflicting_intervals(self, slot: RehearsalSlot) -> list[TimeInterval]:
        """
        Checks for conflicts if the day matches AND the time intervals overlap.
        """
        if self.day_of_week != slot.day_of_week:
            return [] # No conflict if it's not on the right day.

        # Classic interval overlap check: max(starts) < min(ends)
        overlap_start = max(self.start_time, slot.start_time)
        overlap_end = min(self.end_time, slot.end_time)

        if overlap_start < overlap_end:
            # A conflict exists! Return the actual interval of the conflict.
            return [(overlap_start, overlap_end)]
        
        return []

In [3]:
# class ConstraintTransformer(Transformer):
#     def monday(self, _): return DayOfWeekConstraint(day_of_week="monday")
#     def tuesday(self, _): return DayOfWeekConstraint(day_of_week="tuesday")
#     def wednesday(self, _): return DayOfWeekConstraint(day_of_week="wednesday")
#     def thursday(self, _): return DayOfWeekConstraint(day_of_week="thursday")
#     def friday(self, _): return DayOfWeekConstraint(day_of_week="friday")
#     def saturday(self, _): return DayOfWeekConstraint(day_of_week="saturday")
#     def sunday(self, _): return DayOfWeekConstraint(day_of_week="sunday")
#     def day_spec(self, items): return items[0]
#     def unavailability_spec(self, items): return items[0]
#     def conflict_text(self, items): return list(items)
#     def process_conflicts(self, items): return items[0]
    

In [4]:
def check_for_conflict(
    rehearsal: RehearsalSlot,
    dancer_constraints: list[UnavailabilityConstraint]
) -> list[TimeInterval]:
    """Checks for conflicts and returns a list of conflicting intervals."""
    all_conflicts = []
    for constraint in dancer_constraints:
        # print(f"constraint is {constraint}")
        conflicting_intervals = constraint.get_conflicting_intervals(rehearsal)
        all_conflicts.extend(conflicting_intervals)
    return all_conflicts

In [5]:
# def check_constraint_spec(parser, text: str):
#     try:
#         tree = parser.parse(text)
#         return (True, tree, None)
#     except:
#         error_text = analyze_constraint_error(text)
#         return (False, None, error_text)
        
#     def analyze_constraint_error(t):
#         elements = t.split(",")
#         for e in elements:
#             try:
#                 tree = parser.parse(e)
#             except:
#                 return (f"'{e}' in '{t}' is not a valid constraint")
#         return f"Unexpected error in {t}"

In [6]:
from lark import Lark
from lark.exceptions import LarkError # Import the base Lark exception

def check_and_parse_constraints(parser: Lark, text: str) -> tuple[bool, list | None, str | None]:
    """
    Parses a comma-separated string of constraints, validating each one individually.

    Returns:
        A tuple of (success, results, error_message).
        - If successful, (True, [list_of_constraints], None).
        - If failed, (False, None, "error description").
    """
    # Split the input string into individual constraint chunks
    constraint_chunks = [chunk.strip() for chunk in text.split(',') if chunk.strip()]

    success = True
    if not constraint_chunks:
        return (success, [], None) # An empty string is valid

    parsed_constraints = []
    errors = []

    for chunk in constraint_chunks:
        print(f"chunk {chunk}")
        try:
            # We attempt to parse each chunk on its own
            parsed_result = parser.parse(chunk)
            
            # The result of parsing a single spec is a list with one item
            parsed_constraints.extend(parsed_result)

        except LarkError as e:
            # This catches SYNTACTIC errors (e.g., "zulu")
            # Lark's exceptions are very informative!
            print("LarkError")
            success = False
            errors.append(f"Constraint '{chunk}' is not valid. It has a syntax error near '{e.args[0]}'.")
        
        except SemanticValidationError as e:
            # This catches SEMANTIC errors from our transformer (e.g., "th after 25")
            print("SemanticValidationError")
            success = False
            errors.append(f"Constraint '{chunk}' is not valid. It has a semantic error: {e}")
            return (False, None, error_message)
            
        except Exception as e:
            # A general catch-all for any other unexpected errors
            print("catch-all")
            success = False
            errors.append(f"An unexpected error occurred while parsing '{chunk}': {e}")

    # If the loop completes without any exceptions, all chunks were valid.
    if success:
        return (success, parsed_constraints, None)
    return (success, None, errors)

## Grammar Specification

In [7]:
from lark import Lark

constraint_grammar = r"""
    ?start: conflict_text -> process_conflicts

    conflict_text: unavailability_spec ("," unavailability_spec)*

    // An unavailability can now be a simple day or a more complex time-on-day spec
    unavailability_spec: time_on_day_spec | day_spec

    // A simple day spec (e.g., "Monday") becomes a DayOfWeekConstraint
    day_spec: MONDAY | TUESDAY | WEDNESDAY | THURSDAY | FRIDAY | SATURDAY | SUNDAY

    // A time-on-day spec combines a day with a time range
    time_on_day_spec: day_spec time_range

    // --- Time Range Rules ---
    // These define the "until", "after", and "X-Y" formats
    ?time_range: until_range | after_range | explicit_range

    until_range: "until"i time 
               | "before"i time -> build_until_range
    after_range: "after"i time -> build_after_range
    explicit_range: time "-" time -> build_explicit_range

    // --- Time Parsing Rules ---
    // This rule is the key to flexible time parsing.
    // It captures an integer and optional AM/PM marker.
    time: INT -> number_only
        | INT AM_PM -> number_with_ampm

    AM_PM: "am"i | "pm"i

    // --- Day of Week Terminals (Unchanged) ---
    MONDAY:    "monday"i    | "mon"i   | "mo"i | "m"i
    TUESDAY:   "tuesday"i   | "tues"i  | "tu"i
    WEDNESDAY: "wednesday"i | "wed"i   | "we"i | "w"i
    THURSDAY:  "thursday"i  | "thurs"i | "th"i
    FRIDAY:    "friday"i    | "fri"i   | "fr"i | "f"i
    SATURDAY:  "saturday"i  | "sat"i   | "sa"i
    SUNDAY:    "sunday"i    | "sun"i   | "su"i

    %import common.INT
    %import common.WS
    %ignore WS
"""

In [8]:
from lark import Lark, Transformer, v_args
from typing import TypeAlias, Optional

# --- A custom exception for semantic errors ---
class SemanticValidationError(ValueError):
    """Raised when the syntax is valid but the meaning is not."""
    pass


@v_args(inline=True)
class ConstraintTransformer(Transformer):
    # --- Time Normalization and Validation Helper ---
    def _normalize_time(self, hour: int, am_pm: Optional[str] = None) -> int:
        """
        Converts various time formats to a military time integer.
        This is where semantic validation happens!
        """
        hour = int(hour)
        
        if am_pm:
            am_pm = am_pm.lower()
            if not (1 <= hour <= 12):
                raise ValueError(f"Invalid 12-hour format: Hour '{hour}' must be between 1 and 12.")
            if am_pm == 'pm' and hour < 12:
                hour += 12
            elif am_pm == 'am' and hour == 12: # Midnight case
                hour = 0
        else:
            # Handles military time (e.g., 1400) or simple hours (e.g., 2 in "2-4")
            if hour > 24: # This catches your "after 25" case!
                raise SemanticValidationError(f"Invalid 24-hour format: Hour '{hour}' cannot be greater than 24.")
            if hour >= 1 and hour <= 7: # Heuristic for "2-4" meaning PM
                # If you write "w 2-4", you almost certainly mean 2 PM to 4 PM.
                hour += 12
        
        return hour * 100 # Convert to military time format (e.g., 14 -> 1400)

    # --- Time Parsing Methods ---
    def number_only(self, hour_token):
        return self._normalize_time(hour_token.value)

    def number_with_ampm(self, hour_token, am_pm_token):
        return self._normalize_time(hour_token.value, am_pm_token.value)

    # --- Time Range Builders ---
    def build_until_range(self, end_time):
        return (0, end_time) # 0 is the start of the day

    def build_after_range(self, start_time):
        return (start_time, 2359) # 2359 is the end of the day

    def build_explicit_range(self, start_time, end_time):
        if start_time >= end_time:
            raise ValueError(f"Invalid time range: Start time {start_time} must be before end time {end_time}.")
        return (start_time, end_time)

    # --- Constraint Object Creation ---
    def time_on_day_spec(self, day_of_week_obj, time_range_tuple):
        # day_of_week_obj is a DayOfWeekConstraint, we just need its string value
        day_str = day_of_week_obj.day_of_week
        start_time, end_time = time_range_tuple
        return TimeOnDayConstraint(day_str, start_time, end_time)

    # --- Day of Week Rules (now create DayOfWeekConstraint) ---
    def MONDAY(self, _): return DayOfWeekConstraint("monday")
    def TUESDAY(self, _): return DayOfWeekConstraint("tuesday")
    def WEDNESDAY(self, _): return DayOfWeekConstraint("wednesday")
    def THURSDAY(self, _): return DayOfWeekConstraint("thursday")
    def FRIDAY(self, _): return DayOfWeekConstraint("friday")
    def SATURDAY(self, _): return DayOfWeekConstraint("saturday")
    def SUNDAY(self, _): return DayOfWeekConstraint("sunday")

    # --- Structural/Collection Rules ---
    def day_spec(self, day): return day
    def unavailability_spec(self, spec): return spec
    
    @v_args(inline=False)
    def conflict_text(self, items): return list(items)
    
    @v_args(inline=True)
    def process_conflicts(self, conflicts_list): return conflicts_list

## executable code

In [9]:
# Step 3.1: "Compile" the grammar into a parser object
from lark import Lark

print("--- Compiling Grammar ---")
constraint_parser = Lark(constraint_grammar)
print("✅ Grammar compiled successfully.\n")

--- Compiling Grammar ---
✅ Grammar compiled successfully.



In [10]:
# Our example text
constraint_text = "Monday, wed, F, sat"

In [11]:
ok, tree, error_text = check_and_parse_constraints(constraint_parser, "Monday, wed, F, sat")
if ok:
    print(tree.pretty())
else:
    print(error_text)

chunk Monday
catch-all
chunk wed
catch-all
chunk F
catch-all
chunk sat
catch-all
["An unexpected error occurred while parsing 'Monday': 'Tree' object is not iterable", "An unexpected error occurred while parsing 'wed': 'Tree' object is not iterable", "An unexpected error occurred while parsing 'F': 'Tree' object is not iterable", "An unexpected error occurred while parsing 'sat': 'Tree' object is not iterable"]


In [12]:
ok, tree, error_text = check_and_parse_constraints(constraint_parser, "th after 25")
if ok:
    print(tree.pretty())
else:
    print(error_text)

chunk th after 25
catch-all
["An unexpected error occurred while parsing 'th after 25': 'Tree' object is not iterable"]


In [13]:
ok, tree, error_text = check_and_parse_constraints(constraint_parser, "Monday, wed before 12, F, sat")
if ok:
    print(tree.pretty())
else:
    error_text

chunk Monday
catch-all
chunk wed before 12
catch-all
chunk F
catch-all
chunk sat
catch-all


In [14]:
# Let's create an instance of our transformer and use it
print("--- Transforming the AST ---")
transformer = ConstraintTransformer()
output = transformer.transform(tree)
print(output)

--- Transforming the AST ---
None


In [15]:
print(f"✅ Transformation complete!\n")
print(f"Final Python Object Type: {type(output)}")
print(f"Final Python Object Value: {output}")

# Let's check the type of the first item
print(f"Type of first item: {type(output)}")

✅ Transformation complete!

Final Python Object Type: <class 'NoneType'>
Final Python Object Value: None
Type of first item: <class 'NoneType'>


In [16]:
constraint_parser = Lark(constraint_grammar, start='start', parser='lalr', transformer=ConstraintTransformer())
dancer_unavailability_string = "mon, w, friday, m"
parsed_constraints = constraint_parser.parse(dancer_unavailability_string)

# print(f"Input string: '{dancer_unavailability_string}'")
# print("Parsed constraint objects:")
# for c in parsed_constraints:
#     print(f"  - {c}")
# print(f"Type of parsed_constraints: {type(parsed_constraints)}\n")

In [17]:
rehearsal_1 = RehearsalSlot(date(2024, 9, 4), 'wednesday', 1800, 1930) # Should conflict
rehearsal_2 = RehearsalSlot(date(2024, 9, 5), 'thursday', 1900, 2100) # Should NOT conflict

conflicts_1 = check_for_conflict(rehearsal_1, parsed_constraints)
print(f"Checking Rehearsal 1 (Wednesday): Found conflicts: {conflicts_1}")

conflicts_2 = check_for_conflict(rehearsal_2, parsed_constraints)
print(f"Checking Rehearsal 2 (Thursday): Found conflicts: {conflicts_2}")

Checking Rehearsal 1 (Wednesday): Found conflicts: [(1800, 1930)]
Checking Rehearsal 2 (Thursday): Found conflicts: []


In [18]:
# Assuming all the classes (RehearsalSlot, constraints) and the transformer are defined above...

# --- Main Execution Logic ---
try:
    # 1. Instantiate the parser with the new grammar and transformer
    constraint_parser = Lark(constraint_grammar, start='start', parser='lalr', transformer=ConstraintTransformer())

    # 2. Define a complex unavailability string
    dancer_unavailability_string = "mo until 12, w 2-4, fri after 5pm, sunday"

    # 3. Parse the string
    parsed_constraints = constraint_parser.parse(dancer_unavailability_string)

    print(f"Input string: '{dancer_unavailability_string}'")
    print("Parsed constraint objects:")
    for c in parsed_constraints:
        print(f"  - {c}")
    print("\n")
    
    # 4. Define rehearsals to check
    rehearsal_ok = RehearsalSlot(date(2024, 9, 2), 'monday', 1300, 1500) # OK, it's after 1200
    rehearsal_conflict_1 = RehearsalSlot(date(2024, 9, 2), 'monday', 1100, 1230) # CONFLICT, overlaps with "until 1200"
    rehearsal_conflict_2 = RehearsalSlot(date(2024, 9, 4), 'wednesday', 1500, 1700) # CONFLICT, overlaps with 1400-1600
    rehearsal_conflict_3 = RehearsalSlot(date(2024, 9, 8), 'sunday', 1000, 1100) # CONFLICT, unavailable all day Sunday

    # 5. Check for conflicts
    print(f"Checking {rehearsal_ok.day_of_week} at {rehearsal_ok.start_time}:")
    print(f"  -> Conflicts: {check_for_conflict(rehearsal_ok, parsed_constraints)}\n")

    print(f"Checking {rehearsal_conflict_1.day_of_week} at {rehearsal_conflict_1.start_time}:")
    print(f"  -> Conflicts: {check_for_conflict(rehearsal_conflict_1, parsed_constraints)}\n")
    
    print(f"Checking {rehearsal_conflict_2.day_of_week} at {rehearsal_conflict_2.start_time}:")
    print(f"  -> Conflicts: {check_for_conflict(rehearsal_conflict_2, parsed_constraints)}\n")

    print(f"Checking {rehearsal_conflict_3.day_of_week} at {rehearsal_conflict_3.start_time}:")
    print(f"  -> Conflicts: {check_for_conflict(rehearsal_conflict_3, parsed_constraints)}\n")

    # 6. Test the validation
    print("Testing invalid input 'th after 25'...")
    invalid_string = "th after 25"
    constraint_parser.parse(invalid_string)

except Exception as e:
    print(f"Successfully caught expected error: {e}")

Successfully caught expected error: cannot unpack non-iterable Tree object
